In [11]:
import numpy as np
import networkx as nx
import polars as pl
import matplotlib.pyplot as plt
import random
from collections import defaultdict

In [12]:
# reproducible RNG
random.seed(42)
np.random.seed(42)

# STEP 1: Clean up data/rename and prepare for simulation

# load datasets 
nurse_df = pl.read_csv("nurse_population.txt", separator=" ", has_header=True, ignore_errors=True)
pat_df   = pl.read_csv("patient_population.txt",  separator=" ", has_header=True, ignore_errors=True)

# rename header columns to maintain consistency 
nurse_df = nurse_df.rename({"#id": "agent_id"}).with_columns(pl.lit("NURSE").alias("role"))
pat_df   = pat_df.rename({"#id": "agent_id"}).with_columns(pl.lit("PATIENT").alias("role"))
agents_df = pl.concat([nurse_df, pat_df], how="vertical_relaxed")
agents = {r["agent_id"]: {"role": r["role"]} for r in agents_df.iter_rows(named=True)}  

# load and merge visits 
v_cols = {"#Person": "agent_id", "location": "location_id", "StartTime": "t0", "EndTime": "t1"}
v1 = pl.read_csv("visits_m9.txt",       separator=" ", has_header=True, ignore_errors=True).rename(v_cols)
v2 = pl.read_csv("nurse_visits_m9.txt", separator=" ", has_header=True, ignore_errors=True).rename(v_cols)

visits = (
    pl.concat([v1, v2], how="vertical_relaxed")
      .select(["agent_id", "location_id", "t0", "t1"])
      .with_columns([
          pl.col("t0").cast(pl.Int64, strict=False),
          pl.col("t1").cast(pl.Int64, strict=False),
      ])
      .drop_nulls(["t0","t1"])
      .filter(pl.col("t1") > pl.col("t0"))
)

# test to make sure everything has been imported
print(nurse_df)
print(pat_df)
print(agents_df) # nurses and patients together

print(v1) 
print(v2)
print(visits)

shape: (12, 7)
┌──────────┬───────┬─────────┬──────┬──────┬──────┬───────┐
│ agent_id ┆ group ┆ homeloc ┆ fsm0 ┆ fsm1 ┆ demo ┆ role  │
│ ---      ┆ ---   ┆ ---     ┆ ---  ┆ ---  ┆ ---  ┆ ---   │
│ i64      ┆ i64   ┆ i64     ┆ i64  ┆ i64  ┆ i64  ┆ str   │
╞══════════╪═══════╪═════════╪══════╪══════╪══════╪═══════╡
│ 10000    ┆ 1     ┆ 9910000 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10001    ┆ 1     ┆ 9910001 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10002    ┆ 1     ┆ 9910002 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10003    ┆ 1     ┆ 9910003 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10004    ┆ 1     ┆ 9910004 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ …        ┆ …     ┆ …       ┆ …    ┆ …    ┆ …    ┆ …     │
│ 10007    ┆ 2     ┆ 9910007 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10008    ┆ 2     ┆ 9910008 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10009    ┆ 2     ┆ 9910009 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10010    ┆ 2     ┆ 9910010 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
│ 10011    ┆ 2     ┆ 9910011 ┆ 1    ┆ 2    ┆ 1    ┆ NURSE │
└──────────┴───────┴─────

In [13]:
# STEP 2: Convert the given times into days

# seconds → day index
DAY = 24 * 60 * 60
visits_day = (
    visits
    .with_columns([
        (pl.col("t0") // DAY).alias("day"),
        (pl.col("t1") // DAY).alias("day1"),
    ])
    .with_columns(pl.col("day").alias("d"))
    .select(["agent_id", "location_id", "d"])
    .unique()
)

# STEP 3: we have to basically group nurses and patients together based on the day + if they were in the same location

# roster is a dict
# key: (day, location_id)
# value: (list of agent IDs that were present in that location at that day)
roster = {}

# groups "visits" files by day d and location (based on location_id)
for (d, loc), g in visits_day.group_by(["d", "location_id"]):
    # extract ALL agent ids in that (d, location) group --> put them in a list --> store that list in the roster dictionary
    roster[(int(d), str(loc))] = [aid for (aid,) in g.select("agent_id").iter_rows()]

print(roster)

{(12, '972'): [10008], (7, '1'): [10008, 10009, 10007, 10001, 10000, 10002, 10005, 10004, 10003, 10006, 10011, 10010], (7, '976'): [10010], (17, '971'): [10001], (1, '9910006'): [10006], (12, '978'): [10011], (13, '969'): [10000, 10006], (20, '9910004'): [10004], (27, '9910009'): [10009], (4, '9910003'): [10003], (15, '970'): [10007], (22, '90400'): [10002], (7, '90400'): [10009], (28, '9910002'): [10002], (16, '978'): [10011, 10005], (18, '9910004'): [10004], (14, '9910004'): [10004], (16, '975'): [10009, 10003], (14, '9910011'): [10011], (6, '976'): [10004], (0, '9910002'): [10002], (15, '1'): [10001, 10005, 10009, 10002, 10010, 10006, 10007, 10004, 10011, 10000, 10008, 10003], (5, '9910010'): [10010], (11, '971'): [10001, 10007], (19, '9910003'): [10003], (12, '9910006'): [10006], (8, '978'): [10005, 10011], (13, '9910011'): [10011], (29, '972'): [10002], (21, '969'): [10000], (30, '90400'): [10006], (6, '9910009'): [10009], (18, '973'): [10008], (28, '9910010'): [10010], (17, '977'

In [14]:
# STEP 4: preparing for simulation

# use ALL days present in the data; seed at earliest active day 
days = sorted({int(d) for (d, _) in roster.keys()})
EARLY = days[0]

# initialize the states (S/I/R)
agent_ids = set(agents_df["agent_id"].to_list()) # groups all patients and nurses together into agent_ids
state = {aid: "S" for aid in agent_ids} # create a dict that maps each agent to a disease state. initially, everyone starts as out S/susceptible

# creating initial infections
first_day_agents = {a for (d, _), ppl in roster.items() if d == EARLY for a in ppl}
active_agents = {a for (_, _), ppl in roster.items() for a in ppl}

# if there were people on the earliest day, use them as the first people to get infected
# otherwise, if no one was there, use everyone who was active (active_agents)
seed_pool = list(first_day_agents if first_day_agents else active_agents if active_agents else agent_ids)

# randomly infect 3 people/agents. This means their state changes from S to I 
for a in random.sample(seed_pool, min(3, len(seed_pool))):
    state[a] = "I"

# person-to-location parameters
beta_env = 0.02              # sets how infectious contaminated rooms are AKA the transmission rate
shed_rate = {"NURSE": 0.5, "PATIENT": 1.0} # how much each infected person contaminates the rooms 
half_life_days = 45          # how long C.diff spores persist in rooms
decay = 1.0 - (0.5 ** (1.0 / half_life_days))  # daily proportion of spores lost 
recovery = 1/14                 # recovery rate of infected people 


In [ ]:
# STEP 5: finally the simulation

# creates dict that keeps track of how contaminated a room is 
room_load = defaultdict(float)

# creates an empty list so we can collect the simulation daily summary data 
daily_stats = []


# SIMULATION
for d in days:
    # 1) shedding: infected present today add to room load
    for (day_key, room), people in roster.items():
        if day_key != d:
            continue
        add = 0.0
        for a in people:
            if state.get(a) == "I":
                role = agents.get(a, {}).get("role", "PATIENT")
                add += shed_rate.get(role, 1.0)
        if add:
            room_load[room] += add

    # 2) exposure from room environment
    for (day_key, room), people in roster.items():
        if day_key != d:
            continue
        L = room_load.get(room, 0.0)
        if L <= 0:
            continue
        p_inf = 1.0 - np.exp(-beta_env * L)
        for a in people:
            if state[a] == "S" and random.random() < p_inf:
                state[a] = "I"

    # 3) recoveries (SIR); set recovery=0 to get SI
    for a in list(state.keys()):
        if state[a] == "I" and random.random() < recovery:
            state[a] = "R"

    # 4) environmental decay (once per day)
    for r in list(room_load.keys()):
        room_load[r] *= (1.0 - decay)

    # record counts
    S = sum(s == "S" for s in state.values())
    I = sum(s == "I" for s in state.values())
    R = sum(s == "R" for s in state.values())
    daily_stats.append({"day": d, "S": S, "I": I, "R": R})

# RESULTS
epi = pl.DataFrame(daily_stats)

print("Epidemic curve (first 10 days with activity):")
print(epi.head(10)) # printing the first 10 days
print("Epidemic curve (last 10 days with activity):")
print(epi.tail(10)) # printing the last days/end of infection period 

Epidemic curve (first 10 days with activity):
shape: (10, 4)
┌─────┬─────┬─────┬─────┐
│ day ┆ S   ┆ I   ┆ R   │
│ --- ┆ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╪═════╡
│ 0   ┆ 60  ┆ 3   ┆ 0   │
│ 1   ┆ 60  ┆ 3   ┆ 0   │
│ 2   ┆ 58  ┆ 4   ┆ 1   │
│ 3   ┆ 58  ┆ 4   ┆ 1   │
│ 4   ┆ 57  ┆ 4   ┆ 2   │
│ 5   ┆ 55  ┆ 6   ┆ 2   │
│ 6   ┆ 53  ┆ 8   ┆ 2   │
│ 7   ┆ 52  ┆ 8   ┆ 3   │
│ 8   ┆ 52  ┆ 8   ┆ 3   │
│ 9   ┆ 52  ┆ 8   ┆ 3   │
└─────┴─────┴─────┴─────┘
Epidemic curve (last 10 days with activity):
shape: (10, 4)
┌─────┬─────┬─────┬─────┐
│ day ┆ S   ┆ I   ┆ R   │
│ --- ┆ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╪═════╡
│ 22  ┆ 51  ┆ 1   ┆ 11  │
│ 23  ┆ 51  ┆ 1   ┆ 11  │
│ 24  ┆ 51  ┆ 1   ┆ 11  │
│ 25  ┆ 51  ┆ 1   ┆ 11  │
│ 26  ┆ 51  ┆ 1   ┆ 11  │
│ 27  ┆ 51  ┆ 1   ┆ 11  │
│ 28  ┆ 51  ┆ 1   ┆ 11  │
│ 29  ┆ 51  ┆ 1   ┆ 11  │
│ 30  ┆ 51  ┆ 1   ┆ 11  │
│ 31  ┆ 51  ┆ 1   ┆ 11  │
└─────┴─────┴─────┴─────┘


* So I think we can change things to see how different factors affect the spread of C.diff. 
* We could change:
* 1. We can clean the rooms more aka increase decay
* 2. We can change beta_env
etc